In [2]:
!pip install pyodbc



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pyodbc
import pandas as pd
from tabulate import tabulate

# MS SQL Serverga ulanish
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=RASULBEK\\SQLEXPRESS;"
    "Database=MyDatabase;"
    "Trusted_Connection=yes;"
)

# Overdue loans query
query = """
SELECT 
    l.loan_id,
    l.book_id,
    l.member_id,
    l.loan_date,
    l.return_date,
    DATEADD(DAY, 14, l.loan_date) AS expected_return_date,
    DATEDIFF(DAY, DATEADD(DAY, 14, l.loan_date), ISNULL(l.return_date, GETDATE())) AS days_late
FROM loans l
WHERE (l.return_date IS NULL OR l.return_date > DATEADD(DAY, 14, l.loan_date));
"""

df = pd.read_sql(query, conn)

# Tabulate bilan jadval ko‘rinishida chiqarish
print(tabulate(df, headers="keys", tablefmt="psql", showindex=False))


+-----------+-----------+-------------+-------------+---------------+------------------------+-------------+
|   loan_id |   book_id |   member_id | loan_date   | return_date   | expected_return_date   |   days_late |
|-----------+-----------+-------------+-------------+---------------+------------------------+-------------|
|         1 |         1 |           1 | 2023-01-10  | 2023-01-25    | 2023-01-24             |           1 |
|         2 |         3 |           2 | 2023-01-20  | 2023-02-15    | 2023-02-03             |          12 |
|         3 |         5 |           3 | 2023-02-01  | 2023-02-20    | 2023-02-15             |           5 |
|         4 |         6 |           4 | 2023-02-18  | 2023-03-05    | 2023-03-04             |           1 |
|         5 |         7 |           5 | 2023-03-10  |               | 2023-03-24             |         910 |
|         7 |         2 |           1 | 2023-04-20  |               | 2023-05-04             |         869 |
|         8 |      

C:\Users\Rasulbek_Ruzmetov\AppData\Local\Temp\ipykernel_19180\1651445713.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [6]:
import pyodbc
import pandas as pd

# Ulanish
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=RASULBEK\\SQLEXPRESS;"
    "Database=MyDatabase;"
    "Trusted_Connection=yes;"
)

# SQL query
query = "SELECT TOP 20 * FROM loans"
df = pd.read_sql(query, conn)

# Natijani jadval ko‘rinishida chiqarish
from IPython.display import display
display(df)


C:\Users\Rasulbek_Ruzmetov\AppData\Local\Temp\ipykernel_19180\1032528543.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,loan_id,book_id,member_id,loan_date,return_date
0,1,1,1,2023-01-10,2023-01-25
1,2,3,2,2023-01-20,2023-02-15
2,3,5,3,2023-02-01,2023-02-20
3,4,6,4,2023-02-18,2023-03-05
4,5,7,5,2023-03-10,None
5,6,9,6,2023-04-01,2023-04-15
6,7,2,1,2023-04-20,None
7,8,4,2,2023-05-01,None
8,9,8,3,2023-05-12,None
9,10,10,4,2023-06-01,None


In [8]:
import pyodbc
import pandas as pd

# SQL ulanish
conn = pyodbc.connect(
    "Driver={SQL Server};"
    "Server=RASULBEK\\SQLEXPRESS;"
    "Database=MyDatabase;"
    "Trusted_Connection=yes;"
)

query = """
SELECT 
    book_id,
    YEAR(loan_date) AS year,
    MONTH(loan_date) AS month,
    COUNT(*) AS borrow_count
FROM loans
GROUP BY book_id, YEAR(loan_date), MONTH(loan_date)
ORDER BY year, month;
"""

df = pd.read_sql(query, conn)
print(df.head())


   book_id  year  month  borrow_count
0        1  2023      1             1
1        3  2023      1             1
2        5  2023      2             1
3        6  2023      2             1
4        7  2023      3             1


C:\Users\Rasulbek_Ruzmetov\AppData\Local\Temp\ipykernel_19180\2234442920.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Featurelar
df['date'] = pd.to_datetime(df[['year','month']].assign(DAY=1))
df['month_num'] = df['date'].dt.month
df['year_num'] = df['date'].dt.year

X = df[['book_id','month_num','year_num']]
y = df['borrow_count']

# Train/test bo‘lish
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Baholash
y_pred = model.predict(X_test)
print("MAE:", mean_absolute_error(y_test, y_pred))


MAE: 0.0


In [10]:
import datetime

today = datetime.date.today()
next_month = (today.replace(day=1) + pd.DateOffset(months=1))

future_data = pd.DataFrame({
    "book_id": df['book_id'].unique(),
    "month_num": [next_month.month]*len(df['book_id'].unique()),
    "year_num": [next_month.year]*len(df['book_id'].unique())
})

future_data['predicted_borrow'] = model.predict(future_data)

# Eng ko‘p olinadigan top 10 kitob
top_books = future_data.sort_values(by="predicted_borrow", ascending=False).head(10)
print(top_books)


   book_id  month_num  year_num  predicted_borrow
0        1         10      2025               1.0
1        3         10      2025               1.0
2        5         10      2025               1.0
3        6         10      2025               1.0
4        7         10      2025               1.0
5        2         10      2025               1.0
6        9         10      2025               1.0
7        4         10      2025               1.0
8        8         10      2025               1.0
9       10         10      2025               1.0
